<a href="https://colab.research.google.com/github/GitHub-CTO/coursera_applied-data-science-capstone/blob/master/week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Data Science Capstone - Week 3

Segmenting and Clustering Neighborhoods in Toronto

## 0. Import Libraries

In [0]:
import io

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import requests # library to handle URL requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
# html scraper
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

/bin/bash: conda: command not found


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

/bin/bash: conda: command not found


In [4]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

/bin/bash: conda: command not found


In [5]:
print('Libraries imported.')

Libraries imported.


## 1. Load Neighbourhoods in Toronto (Postal Codes start with M)

In [0]:
# Get the wiki web page

toronto_postal_codes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_postal_codes_html = requests.get(toronto_postal_codes_url).text
# print(toronto_postal_codes_html)

In [0]:
# Parse html tags to extract information later

toronto_postal_codes_soap = BeautifulSoup(toronto_postal_codes_html)
# print(toronto_postal_codes_soap.prettify())

In [0]:
# Find the table with list of postal codes

postal_codes = toronto_postal_codes_soap.find('table', class_='wikitable sortable')

In [0]:
# Column names used in dataframes in this notebook

COL_NAME_POSTAL_CODE = 'PostalCode'
COL_NAME_BOROUGH = 'Borough'
COL_NAME_NEIGHBOURHOOD = 'Neighbourhood'
COL_NAME_LATITUDE = 'Latitude'
COL_NAME_LONGITUDE = 'Longitude'

COL_NAME_VENUE = 'Venue' 
COL_NAME_VENUE_LATITUDE = 'Venue Latitude'
COL_NAME_VENUE_LONGITUDE = 'Venue Longitude'
COL_NAME_VENUE_CATEGORY = 'Venue Category'

COL_NAME_CLUSTER_LABELS = 'Cluster Labels'


# Special values

COL_VALUE_NOT_ASSIGNED = 'Not assigned'

In [10]:
# Parse table data and put them into DataFrame
#
# Notes:
# 1. We don't use the table's column headings here because
#    the wiki uses 'Postcode' but the instructions ask for 'PostalCode'
# 2. Not assigned boroughs are ignored
# 3. A not assigned neigbourhood will have the same name as its borough

columns = [COL_NAME_POSTAL_CODE, COL_NAME_BOROUGH, COL_NAME_NEIGHBOURHOOD]
neighbourhoods = pd.DataFrame(columns=columns)

for row in postal_codes.find_all('tr'):
    cells = [cell.text.strip() for cell in row.find_all('td')]
    
    if len(cells) == 3:
        postal_code = cells[0]
        borough = cells[1]
        neighbourhood = cells[2]
    
        if borough != COL_VALUE_NOT_ASSIGNED:
            # only known boroughs are considered
            neighbourhoods = neighbourhoods.append({
                    COL_NAME_POSTAL_CODE: postal_code,
                    COL_NAME_BOROUGH: borough,
                    # not assigned neighbourhood will have the same name as its borough
                    COL_NAME_NEIGHBOURHOOD: borough if neighbourhood == COL_VALUE_NOT_ASSIGNED else neighbourhood 
                },
                ignore_index=True
            )

print(neighbourhoods.shape)
neighbourhoods.head(10)

(211, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [11]:
# Group neighbourhoods (separated with commas) of the same postal code into one row

neighbourhoods_normalized = pd.DataFrame(columns=columns)

for group_by in neighbourhoods.groupby([COL_NAME_POSTAL_CODE, COL_NAME_BOROUGH]):
    postal_code = group_by[0][0];

    borough = group_by[0][1];

    # sort all neighbourhoods by name before combining them into one string separated with commas
    neigbourhoods_by_borough = group_by[1].sort_values(COL_NAME_NEIGHBOURHOOD)
    neighbourhood = ', '.join(neigbourhoods_by_borough[COL_NAME_NEIGHBOURHOOD].values)

    neighbourhoods_normalized = neighbourhoods_normalized.append({
            COL_NAME_POSTAL_CODE: postal_code,
            COL_NAME_BOROUGH: borough,
            COL_NAME_NEIGHBOURHOOD: neighbourhood 
        },
        ignore_index=True
    )

neighbourhoods_normalized.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
# Answer to the number of rows of neighbourhoods dataframe
# after we group all neighbourhoods of the same postal code

print(neighbourhoods_normalized.shape)

(103, 3)


## 2. Load the Latitude and Longitude for Each Postal Code

In [13]:
# We are going to use the reloaded latitudes and longitudes for Toronto's postal codes

GEOSPATIAL_COORDINATES_URL = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"

csv = requests.get(GEOSPATIAL_COORDINATES_URL).content
geospatial_coordinates = pd.read_csv(io.StringIO(csv.decode('utf-8')))

geospatial_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
neighbourhoods_geo_coordinates = neighbourhoods_normalized

# merge geo coordinates with neighbourhood data to add latitude/longitude for each neighborhood
neighbourhoods_geo_coordinates = neighbourhoods_geo_coordinates.join(
    geospatial_coordinates.set_index('Postal Code'), on=COL_NAME_POSTAL_CODE
)

neighbourhoods_geo_coordinates.shape

(103, 5)

In [15]:
# Answer

neighbourhoods_geo_coordinates

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## 3. Explore and Cluster the Neighborhoods in Toronto

#### We limit our analysis in City of Toronto

In [16]:
# Only boroughs that contain the word Toronto

toronto_neigbourhoods = neighbourhoods_geo_coordinates[
    neighbourhoods_geo_coordinates[COL_NAME_BOROUGH]
    .str.contains(pat = 'Toronto')
]

toronto_neigbourhoods = toronto_neigbourhoods.reset_index(drop=True)

toronto_neigbourhoods

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


#### Create a map of Toronto with neighborhoods by postal codes superimposed on top.

In [17]:
# Find Toronto's latitude and longitude to plot Toronto map/area

geolocator = Nominatim(user_agent="to_explorer")
toronto_location = geolocator.geocode('Toronto, ON', timeout=10)

print('The geograpical coordinate of Toronto are {}, {}.'.format(
    toronto_location.latitude,
    toronto_location.longitude
))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# Create a map of Toronto with postal code locations superimposed on top

toronto_map = folium.Map(location=[toronto_location.latitude, toronto_location.longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(
    toronto_neigbourhoods[COL_NAME_LATITUDE],
    toronto_neigbourhoods[COL_NAME_LONGITUDE],
    toronto_neigbourhoods[COL_NAME_BOROUGH],
    toronto_neigbourhoods[COL_NAME_NEIGHBOURHOOD]):
    
    label = '{} : {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Let's create a function to repeat the same process to all the neighborhoods in City of Toronto

In [0]:
def getNearbyVenues(postal_codes, latitudes, longitudes, radius=500):
    CLIENT_ID = 'H3GCKKX5W2CDBZWK1GF5WNVOVZEOXYMP2OJIBWQ14J1QNDTX' # your Foursquare ID
    CLIENT_SECRET = 'Z1LZ5KLUBPXOV3TIYS5FJL45FH45EK4A45F40B1YEVCJKB33' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    LIMIT = 100
    
    # define the dataframe columns
    column_names = [
        COL_NAME_POSTAL_CODE, 
        COL_NAME_LATITUDE, 
        COL_NAME_LONGITUDE, 
        COL_NAME_VENUE, 
        COL_NAME_VENUE_LATITUDE, 
        COL_NAME_VENUE_LONGITUDE, 
        COL_NAME_VENUE_CATEGORY
    ]

    # instantiate the dataframe
    nearby_venues = pd.DataFrame(columns=column_names)
    
    for postal_code, lat, lng in zip(postal_codes, latitudes, longitudes):
        print(postal_code)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for v in results:
            nearby_venues = nearby_venues.append({
                    COL_NAME_POSTAL_CODE: postal_code,
                    COL_NAME_LATITUDE: lat,
                    COL_NAME_LONGITUDE: lng,
                    COL_NAME_VENUE: v['venue']['name'],
                    COL_NAME_VENUE_LATITUDE: v['venue']['location']['lat'],
                    COL_NAME_VENUE_LONGITUDE: v['venue']['location']['lng'],
                    COL_NAME_VENUE_CATEGORY: v['venue']['categories'][0]['name']
                },
                ignore_index=True
            )
    
    return(nearby_venues)

#### Let's now find all venues in City of Toronto

In [20]:
toronto_venues = getNearbyVenues(
    postal_codes=toronto_neigbourhoods[COL_NAME_POSTAL_CODE],
    latitudes=toronto_neigbourhoods[COL_NAME_LATITUDE],
    longitudes=toronto_neigbourhoods[COL_NAME_LONGITUDE]
)

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


In [21]:
print(toronto_venues.shape)
toronto_venues.head(10)

(1700, 7)


,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,M4K,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,M4K,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,M4K,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,M4K,43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
9,M4K,43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant


#### Let's check how many venues were returned for each neighborhood

In [22]:
toronto_venues.groupby(COL_NAME_POSTAL_CODE)[COL_NAME_VENUE].count()

PostalCode
M4E      4
M4K     42
M4L     21
M4M     41
M4N      4
M4P      8
M4R     20
M4S     34
M4T      2
M4V     15
M4W      5
M4X     46
M4Y     84
M5A     50
M5B    100
M5C    100
M5E     56
M5G     84
M5H    100
M5J    100
M5K    100
M5L    100
M5N      1
M5P      5
M5R     24
M5S     34
M5T    100
M5V     16
M5W     96
M5X    100
M6G     15
M6H     14
M6J     65
M6K     22
M6P     23
M6R     15
M6S     37
M7Y     17
Name: Venue, dtype: int64

#### Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues[COL_NAME_VENUE_CATEGORY].unique())))

# Top 10 categories
toronto_venues[COL_NAME_VENUE_CATEGORY].value_counts()[:10]

There are 234 uniques categories.


Coffee Shop            144
Café                    87
Restaurant              53
Italian Restaurant      47
Bakery                  42
Hotel                   39
Bar                     37
Park                    34
Pizza Place             34
Japanese Restaurant     25
Name: Venue Category, dtype: int64

### Analyze Each Postal Code Area

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[[COL_NAME_VENUE_CATEGORY]], prefix="", prefix_sep="")

# add PostalCode column back to dataframe
toronto_onehot[COL_NAME_POSTAL_CODE] = toronto_venues[COL_NAME_POSTAL_CODE] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1700, 235)


,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by postal code and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby(COL_NAME_POSTAL_CODE).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 235)


,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.

#### Let's print each postal code along with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in toronto_grouped[COL_NAME_POSTAL_CODE]:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped[COL_NAME_POSTAL_CODE] == hood].T.reset_index()
    temp.columns = [COL_NAME_VENUE,'freq']
    temp = temp.iloc[1:] # remove the row of Neighborhood (Neighborhood column before transpose operation)
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                      Venue  freq
0         Health Food Store  0.25
1              Neighborhood  0.25
2                       Pub  0.25
3                     Trail  0.25
4  Mediterranean Restaurant  0.00


----M4K----
                    Venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----M4L----
              Venue  freq
0              Park  0.10
1    Sandwich Place  0.10
2           Brewery  0.05
3  Sushi Restaurant  0.05
4         Pet Store  0.05


----M4M----
                 Venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----M4N----
                        Venue  freq
0  Construction & Landscaping  0.25
1                    Bus Line  0.25
2                        Park  0.25
3                 Swim School  0.25
4                Neighborh

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = [COL_NAME_POSTAL_CODE]
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted[COL_NAME_POSTAL_CODE] = toronto_grouped[COL_NAME_POSTAL_CODE]

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(toronto_venues_sorted.shape)
toronto_venues_sorted.head()

(38, 11)


,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Trail,Pub,Neighborhood,Cosmetics Shop,Coworking Space,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Restaurant
2,M4L,Park,Sandwich Place,Pet Store,Ice Cream Shop,Brewery,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Steakhouse
3,M4M,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Chinese Restaurant,Bookstore,Fish Market,Brewery,Latin American Restaurant
4,M4N,Construction & Landscaping,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Cluster Postal Codes

Run *k*-means to cluster the postal codes into 5 clusters.

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(COL_NAME_POSTAL_CODE, 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 4, 1, 1, 1, 3, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each postal code.

In [30]:
# add clustering labels
if COL_NAME_CLUSTER_LABELS in toronto_venues_sorted.columns:
    # We just update the cluster labels column if the column has been inserted
    # like when we rerun this cell manually
    toronto_venues_sorted[COL_NAME_CLUSTER_LABELS] = kmeans.labels_
else:
    toronto_venues_sorted.insert(0, COL_NAME_CLUSTER_LABELS, kmeans.labels_)

toronto_merged = toronto_neigbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index(COL_NAME_POSTAL_CODE), on=COL_NAME_POSTAL_CODE)

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(38, 16)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhood,Cosmetics Shop,Coworking Space,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Sandwich Place,Pet Store,Ice Cream Shop,Brewery,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Chinese Restaurant,Bookstore,Fish Market,Brewery,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Construction & Landscaping,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Finally, let's visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[toronto_location.latitude, toronto_location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
    toronto_merged[COL_NAME_LATITUDE],
    toronto_merged[COL_NAME_LONGITUDE],
    toronto_merged[COL_NAME_NEIGHBOURHOOD],
    toronto_merged[COL_NAME_CLUSTER_LABELS]):
    # print(poi, lat, lon, cluster)
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster 1

In [32]:
toronto_merged.loc[
    toronto_merged[COL_NAME_CLUSTER_LABELS] == 0,
    toronto_merged.columns[[0, 1, 2] + list(range(5, toronto_merged.shape[1]))]
]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,0,Health Food Store,Trail,Pub,Neighborhood,Cosmetics Shop,Coworking Space,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Cluster 2

In [33]:
toronto_merged.loc[
    toronto_merged[COL_NAME_CLUSTER_LABELS] == 1,
    toronto_merged.columns[[0, 1, 2] + list(range(5, toronto_merged.shape[1]))]
]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,East Toronto,"Riverdale, The Danforth West",1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",1,Park,Sandwich Place,Pet Store,Ice Cream Shop,Brewery,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Steakhouse
3,M4M,East Toronto,Studio District,1,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Chinese Restaurant,Bookstore,Fish Market,Brewery,Latin American Restaurant
5,M4P,Central Toronto,Davisville North,1,Sandwich Place,Clothing Store,Gym,Park,Breakfast Spot,Hotel,Food & Drink Shop,Dance Studio,Doner Restaurant,Donut Shop
6,M4R,Central Toronto,North Toronto West,1,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Dessert Shop,Cosmetics Shop,Park,Chinese Restaurant
7,M4S,Central Toronto,Davisville,1,Sandwich Place,Dessert Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Sushi Restaurant,Thai Restaurant,Coffee Shop,Gourmet Shop
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",1,Pub,Coffee Shop,Liquor Store,American Restaurant,Supermarket,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Restaurant
11,M4X,Downtown Toronto,"Cabbagetown, St. James Town",1,Coffee Shop,Pizza Place,Italian Restaurant,Café,Bakery,Pub,Park,Restaurant,Flower Shop,Indian Restaurant
12,M4Y,Downtown Toronto,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel
13,M5A,Downtown Toronto,"Harbourfront, Regent Park",1,Coffee Shop,Bakery,Park,Pub,Café,Mexican Restaurant,Restaurant,Breakfast Spot,Theater,Farmers Market


Cluster 3

In [34]:
toronto_merged.loc[
    toronto_merged[COL_NAME_CLUSTER_LABELS] == 2,
    toronto_merged.columns[[0, 1, 2] + list(range(5, toronto_merged.shape[1]))]
]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,Central Toronto,Roselawn,2,Garden,Yoga Studio,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Cluster 4

In [35]:
toronto_merged.loc[
    toronto_merged[COL_NAME_CLUSTER_LABELS] == 3,
    toronto_merged.columns[[0, 1, 2] + list(range(5, toronto_merged.shape[1]))]
]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,Central Toronto,"Moore Park, Summerhill East",3,Playground,Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Cluster 5

In [36]:
toronto_merged.loc[
    toronto_merged[COL_NAME_CLUSTER_LABELS] == 4,
    toronto_merged.columns[[0, 1, 2] + list(range(5, toronto_merged.shape[1]))]
]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,Central Toronto,Lawrence Park,4,Construction & Landscaping,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,M4W,Downtown Toronto,Rosedale,4,Park,Playground,Trail,Building,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West",4,Bus Line,Park,Sushi Restaurant,Jewelry Store,Trail,Yoga Studio,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
